In [34]:
!pip install sqlalchemy-views

In [1]:
import sys
sys.path.append('/Users/robertcampbell/sqlalchemy-tutorial/')
sys.path
from typing import List
from sqlalchemy import select, func, MetaData, Table, and_
from sqlalchemy.orm import aliased
from sqlalchemy.sql import text
import pandas as pd
from schema.schema import Transfer, Event, Fixture, Player, Statistic
from main import engine, session
from sqlalchemy_views import CreateView, DropView

In [2]:
statement = text("SELECT * FROM denorm_stats")

with engine.connect() as conn:
    results = conn.execute(statement)

df = pd.DataFrame(results)

In [4]:
df.iloc[20:,:]

,start_time,id,home_team_id,away_team_id,home_goals,away_goals,H_GF_AT_HOME,H_GA_AT_HOME,H_GA_ON_THE_ROAD,H_GF_ON_THE_ROAD,...,away_midfielders_cumulative_average_dribble_success_percentage,away_defenders_cumulative_average_passing_percentage,away_defenders_cumulative_duels_won_percentage,away_defenders_cumulative_rating,away_defenders_cumulative_average_total_passes,away_defenders_cumulative_average_key_passes,away_defenders_cumulative_average_interceptions,away_defenders_cumulative_average_total_tackles,away_defenders_cumulative_average_total_blocks,away_defenders_cumulative_average_dribble_success_percentage
20,2022-08-20 07:30:00.000000,867974,47,39,1,0,4.0,1.0,2.0,2.0,...,64.166667,88.068504,71.924603,6.900000,269.000000,3.000000,5.000000,6.000000,1.000000,83.333333
21,2022-08-20 10:00:00.000000,867969,36,55,3,2,2.0,2.0,0.0,0.0,...,NaN,84.242051,58.333333,6.525000,205.000000,1.000000,2.000000,1.000000,NaN,50.000000
22,2022-08-20 10:00:00.000000,867968,45,65,1,1,0.0,1.0,2.0,1.0,...,83.333333,82.021116,46.296296,6.700000,85.000000,NaN,2.000000,3.000000,4.000000,50.000000
23,2022-08-20 10:00:00.000000,867971,46,41,1,2,2.0,2.0,4.0,2.0,...,100.000000,79.430860,45.222222,6.340000,173.000000,4.000000,7.000000,4.000000,2.000000,66.666667
24,2022-08-20 10:00:00.000000,867967,52,66,3,1,0.0,2.0,1.0,1.0,...,72.222222,83.773562,53.080808,6.750000,262.000000,1.000000,6.000000,3.000000,1.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2023-05-28 11:30:00.000000,868319,49,34,1,1,19.0,18.0,28.0,18.0,...,72.043651,78.277939,60.110141,7.109722,188.611111,3.666667,4.666667,6.333333,2.687500,86.538462
376,2023-05-28 11:30:00.000000,868320,52,65,1,1,20.0,22.0,26.0,19.0,...,72.051282,72.623974,62.882795,6.742593,120.888889,1.750000,4.764706,7.166667,2.941176,83.888889
377,2023-05-28 11:30:00.000000,868318,55,50,1,0,34.0,18.0,28.0,23.0,...,75.844444,90.049734,62.220393,7.025926,296.055556,2.000000,2.666667,4.444444,1.812500,77.111111
378,2023-05-28 11:30:00.000000,868322,63,47,1,4,25.0,33.0,41.0,22.0,...,74.861111,84.981527,59.007470,6.751852,174.000000,1.833333,3.437500,5.666667,2.214286,87.500000


In [105]:
fixture = aliased(Fixture)

In [106]:
h_gf_at_home = select(func.sum(Fixture.home_goals).label("H_GF_AT_HOME"))\
    .where(and_((Fixture.home_team_id == fixture.home_team_id, fixture.start_time < Fixture.start_time)))\
    .subquery()
h_ga_at_home = select(func.sum(Fixture.away_goals).label("H_GA_AT_HOME"))\
    .where(and_(Fixture.home_team_id == fixture.home_team_id, fixture.start_time < Fixture.start_time)) \
    .subquery()
h_ga_on_the_road = select(func.sum(Fixture.home_goals).label("H_GA_ON_THE_ROAD"))\
    .where(Fixture.away_team_id == fixture.home_team_id)\
    .subquery()
h_gf_on_the_road = select(func.sum(Fixture.away_goals).label("H_GF_ON_THE_ROAD"))\
    .where(Fixture.away_team_id == fixture.home_team_id)\
    .subquery()
a_gf_at_home = select(func.sum(Fixture.home_goals).label("A_GF_AT_HOME"))\
    .where(Fixture.home_team_id == fixture.away_team_id)\
    .subquery()
a_ga_at_home = select(func.sum(Fixture.away_goals).label("A_GF_AT_HOME"))\
    .where(Fixture.home_team_id == fixture.away_team_id)\
    .subquery()
a_ga_on_the_road = select(func.sum(Fixture.home_goals).label("A_GA_ON_THE_ROAD"))\
    .where(Fixture.away_team_id == fixture.away_team_id)\
    .subquery()
a_gf_on_the_road = select(func.sum(Fixture.away_goals).label("A_GF_ON_THE_ROAD"))\
    .where(Fixture.away_team_id == fixture.away_team_id)\
    .subquery()

ArgumentError: SQL expression for WHERE/HAVING role expected, got (<sqlalchemy.sql.elements.BinaryExpression object at 0x1659ed880>, <sqlalchemy.sql.elements.BinaryExpression object at 0x167ae1580>).

In [100]:
cum_def = select(fixture.start_time, 
             fixture.home_team_id, 
             fixture.away_team_id, 
             fixture.home_goals, 
             fixture.away_goals, 
             h_gf_at_home,
             h_ga_at_home,
             h_ga_on_the_road,
             h_gf_on_the_road,
             a_gf_at_home,
             a_ga_at_home,
             a_ga_on_the_road,
             a_gf_on_the_road
             )
cum_table = Table('cumulative_goals', MetaData())
cum_view = CreateView(cum_def, cum_table)


In [101]:
with engine.connect() as conn:
    def_res = conn.execute(cum_def)

/var/folders/nx/l7sd1gss2ksd87brx4b4blxh0000gn/T/ipykernel_4429/337623608.py:2: SAWarning: SELECT statement has a cartesian product between FROM element(s) "anon_4", "anon_2", "anon_7", "anon_1", "anon_8", "anon_5", "anon_6", "anon_3" and FROM element "fixture".  Apply join condition(s) between each element to resolve.
  def_res = conn.execute(cum_def)


In [102]:
pd.DataFrame(def_res)

,start_time,home_team_id,away_team_id,home_goals,away_goals,H_GF_AT_HOME,H_GA_AT_HOME,H_GA_ON_THE_ROAD,H_GF_ON_THE_ROAD,A_GF_AT_HOME,A_GF_AT_HOME_1,A_GA_ON_THE_ROAD,A_GF_ON_THE_ROAD
0,2022-08-05 15:00:00,52,42,0,2,11799,8797,11799,8797,11799,8797,11799,8797
1,2022-08-06 07:30:00,36,40,2,2,11799,8797,11799,8797,11799,8797,11799,8797
2,2022-08-06 10:00:00,35,66,2,0,11799,8797,11799,8797,11799,8797,11799,8797
3,2022-08-06 10:00:00,63,39,2,1,11799,8797,11799,8797,11799,8797,11799,8797
4,2022-08-07 09:00:00,46,55,2,2,11799,8797,11799,8797,11799,8797,11799,8797
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2023-05-28 11:30:00,45,35,1,0,11799,8797,11799,8797,11799,8797,11799,8797
376,2023-05-28 11:30:00,63,47,1,4,11799,8797,11799,8797,11799,8797,11799,8797
377,2023-05-28 11:30:00,46,48,2,1,11799,8797,11799,8797,11799,8797,11799,8797
378,2023-05-28 11:30:00,33,36,2,1,11799,8797,11799,8797,11799,8797,11799,8797


AttributeError: 'CreateColumn' object has no attribute 'c'

In [67]:
defender_stats = Table("defender_stats", MetaData())

In [71]:
for column in cum_view.columns:
    print(column)

start_time DATETIME NOT NULL
home_team_id INTEGER NOT NULL
away_team_id INTEGER NOT NULL
home_goals INTEGER
away_goals INTEGER


AttributeError: Neither 'ColumnClause' object nor 'Comparator' object has an attribute 'system'

In [68]:
session.query(defender_stats).all()

InvalidRequestError: Query contains no columns with which to SELECT from.

In [43]:
cum_table.join(
    
)

SELECT 
FROM cumulative_goals


# TABLES Tutorial

In [73]:
import sqlalchemy as sa

In [74]:
tbl = sa.Table("defender_stats", sa.MetaData(), autoload_with=engine)

NoSuchTableError: defender_stats

In [75]:

with engine.connect() as conn:
    rows = conn.execute(sa.select(tbl))

NameError: name 'tbl' is not defined

ArgumentError: expected schema argument to be a string, got <class 'sqlalchemy.engine.base.Engine'>.